# Imports

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss
import pandas as pd
import numpy as np, glob
import pickle
import os

# Non-parametric bootstrap

In [ ]:
from tqdm.notebook import tqdm, trange

def bootstrap_stat(Y_true,Y_pred):
    np.random.seed(42)
    idxs=[np.arange(len(y_true)) for y_true in Y_true]
    aucs_final,logloss_final=[],[]
    for i in trange(1000,desc='bootstrap'):
        aucs=[]
        log_losses=[]
        idxs_new=[np.random.choice(idx,len(idx),replace=True) for idx in idxs]
        for idx,y_true,y_pred in zip(idxs_new,Y_true,Y_pred):
            y_true,y_pred=y_true[idx],y_pred[idx]
            aucs.append(roc_auc_score(y_true,y_pred))
            log_losses.append(log_loss(y_true,y_pred))
        aucs_final.append(np.mean(aucs))
        logloss_final.append(np.mean(log_losses))
    aucs=[]
    log_losses=[]
    for y_true,y_pred in zip(Y_true,Y_pred):
        aucs.append(roc_auc_score(y_true,y_pred))
        log_losses.append(log_loss(y_true,y_pred))
    return np.mean(aucs),np.mean(log_losses),aucs_final,logloss_final
        
boot_res=dict()
for f in tqdm(glob.glob("../../results/*.pkl"),desc="algo"):
    if f not in list(boot_res.keys()): boot_res[f]=dict()
    res_=pd.read_pickle(f)
    for k in tqdm(list(res_.keys()),desc="type", position=1, leave=True):
        Y_true=[np.array(res_[k][i]['y_true']) for i in (range(1,11) if isinstance(res_[k],dict) else range(10))]
        Y_pred=[np.array(res_[k][i]['y_pred']) for i in (range(1,11) if isinstance(res_[k],dict) else range(10))]
        if k not in list(boot_res[f].keys()): boot_res[f][k]=bootstrap_stat(Y_true,Y_pred)

In [ ]:
pickle.dump(boot_res,open('../../bootstrap_results.pkl','wb'))

# Display Results

In [ ]:
boot_res2={os.path.basename(k).replace(".pkl",""):{k2:round(pd.Series([v2[0],np.std(v2[2]),v2[1],np.std(v2[3])]),3).values for k2,v2 in v.items()} for k,v in boot_res.items()}

In [ ]:
pd.DataFrame(boot_res2).applymap(lambda x: f"{x[0]}±{x[1]}" if str(x)!="nan" else "")

In [ ]:
pd.DataFrame(boot_res2).applymap(lambda x: f"{x[2]}±{x[3]}" if str(x)!="nan" else "")